In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_curve, roc_auc_score
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

### Задача 1
Обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта

### Задача 2
При обучении моделей обязательно использовать кроссвалидацию

### Задача 3
Вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)

In [2]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [3]:
df = pd.read_csv('./input/train_case2.csv', sep = ';').fillna(' ').sample(frac=1)
df.shape

(70000, 13)

In [4]:
target_inx = 12
target = df.iloc[:, -1]
target.shape

(70000,)

In [5]:
train = df.iloc[:, :-1]
train.shape

(70000, 12)

In [6]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(train, target, random_state=6)

In [7]:
continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [8]:
feats = FeatureUnion(continuos_transformers + cat_transformers + base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-0.17383452,  0.08328586, -0.63527261, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.57163995, -1.51274396, -1.25961961, ...,  0.        ,
         0.        ,  1.        ],
       [-1.67450704,  0.32882891,  0.68279328, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.27912225, -1.26720091,  1.1683965 , ...,  0.        ,
         0.        ,  1.        ],
       [ 1.04404388, -1.02165786,  1.37651217, ...,  0.        ,
         0.        ,  1.        ],
       [-1.85925506,  1.06545806,  3.45766884, ...,  0.        ,
         0.        ,  0.        ]])

In [9]:
gb_model = GradientBoostingRegressor(loss='huber', 
                                     learning_rate=0.25, 
                                     n_estimators=650,  
                                     min_samples_leaf=1,
                                     min_weight_fraction_leaf=0.12, 
                                     random_state=23)

rf_model = RandomForestRegressor(min_samples_split = 20, 
                                 max_depth=41,
                                 n_estimators=600, 
                                 min_samples_leaf=2,
                                 min_weight_fraction_leaf=0.005, 
                                 random_state=23)

etr_model = ExtraTreesRegressor(n_estimators=40, 
                                max_depth= 5,
                                max_leaf_nodes= 9,
                                bootstrap = True,
                                random_state=23)
# results = pd.dataframe()
models = ['GradientBoostingRegressor', 'RandomForestRegressor', 'ExtraTreesRegressor']
col = ['model_name',  'model', 'auc_scores', 'precision', 'recall', 'fscore', 'cv_score', 'cv_score_std']

# models = [gb_model]
df_models = pd.DataFrame(columns=col)
df_models['model_name'] = ['GradientBoostingRegressor', 'RandomForestRegressor', 'ExtraTreesRegressor']
df_models['model'] = [gb_model, rf_model, etr_model]

df_models

,model_name,model,auc_scores,precision,recall,fscore,cv_score,cv_score_std
0,GradientBoostingRegressor,"GradientBoostingRegressor(learning_rate=0.25, ...",NaN,NaN,NaN,NaN,NaN,NaN
1,RandomForestRegressor,"RandomForestRegressor(max_depth=41, min_sample...",NaN,NaN,NaN,NaN,NaN,NaN
2,ExtraTreesRegressor,"ExtraTreesRegressor(bootstrap=True, max_depth=...",NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
b = 1
for model in models:
    classifier = Pipeline([
        ('features',feats),
        ('classifier', df_models.loc[df_models['model_name'] == model, 'model'].values[0]),
    ])
    
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=2, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)
    
    #обучим пайплайн на всем тренировочном датасете
    classifier.fit(X_train, y_train)
    y_score = classifier.predict(X_test)
    auc_score = roc_auc_score(y_true=y_test, y_score=y_score)
    
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1 + b ** 2)*(precision * recall) / (b**2*precision + recall)
    
    ix = np.argmax(fscore)
    
    df_models.loc[df_models['model_name'] == model, col[2:]] = (auc_score, precision[ix], recall[ix], fscore[ix], cv_score, cv_score_std)   


In [11]:
df_models.drop('model', 1)

,model_name,auc_scores,precision,recall,fscore,cv_score,cv_score_std
0,GradientBoostingRegressor,0.794627,0.661775,0.834023,0.737982,0.793779,0.000986
1,RandomForestRegressor,0.799203,0.681753,0.813232,0.741711,0.799327,0.001216
2,ExtraTreesRegressor,0.71661,0.579257,0.904549,0.706246,0.723781,0.003272


### Задача 4
Сделать выводы о том, какая модель справилась с задачей лучше других

Модель RandomForestRegressor справилась лучше остальных по большинству показателей уступая конкурентам только по полноте, 
что говорит большем количестве ошибок второго рода.

### Задача 5*
Какой график (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого, например, 1 к 1000).
p.s.  
В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно).
Допустим, у нас две модели:

первая помечает 100 объектов как класс 1, но TP = 90
вторая помечает 1000 объектов как класс 1, но TP такой же - 90
Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

In [12]:
# первая модель
TP = 90
FP = 10
TN = 999890
FN = 10

precision = TP / (TP + FP)
recall = TP / (TP + FN)

TPR = TP / (TP + FN)
FPR = FP / (FP + TN)

print(f'precision = {precision}, recall = {recall}, TPR = {TPR}, FPR = {FPR:.6f}')

precision = 0.9, recall = 0.9, TPR = 0.9, FPR = 0.000010


In [13]:
# первая модель
TP = 90
FP = 910
TN = 998990
FN = 10

precision = TP / (TP + FP)
recall = TP / (TP + FN)

TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
print(f'precision = {precision}, recall = {recall}, TPR = {TPR}, FPR = {FPR:.6f}')

precision = 0.09, recall = 0.9, TPR = 0.9, FPR = 0.000910


Как можно заметить использование roc_auc_curve на данных с дисбалансом классов не покажет разницы между различными моделями.
В следствии чего на несбалансированных данных предпочтительней использовать precision_recall_curve.

В данном примере лучше будет использовать первую модель так как у нее выше точность, а полнота в обеих моделях одинакова.
На мой взгляд  precision_recall_curve интуитивно понятней.